In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Carregando Pacotes

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
from tensorboard.plugins.hparams import api as hp
from pathlib import Path
import sklearn.metrics
import sklearn
import itertools
import io
import tensorflow_addons as tfa

2022-05-10 21:21:48.777689: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
print(tf.__version__)

2.4.1


# Paramêtros

In [4]:
# Classes
num_classes = 7

# Epochs
n_epochs = 1000
n_epochs_ini = 100

# Tamanho imagem
img_rows, img_cols = 224, 224

input_shape = (img_rows, img_cols, 3)

#batch_size = 64

strategy = tf.distribute.MirroredStrategy()

BATCH_SIZE_PER_REPLICA = 16
batch_size = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA


# seed
seed = 42

path_dataset = Path('/scratch/dpfurtado/backup_home/corais/1_classificacao_train_50/dataset/1_train_50')
path_train = path_dataset/'train'
path_train_dev = path_dataset/'train-dev'

nome_modelo = 'resnet_plc_full_transfer'


logs_dir = os.path.join('./logs/', nome_modelo)

!rm -r {logs_dir}

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
rm: cannot remove './logs/resnet_plc_full_transfer': No such file or directory


2022-05-10 21:21:50.524184: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-10 21:21:50.526594: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-10 21:21:51.613521: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 312.92GiB/s
2022-05-10 21:21:51.614218: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:af:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 312.92GiB/s
2022-05-10 21:21:51.614283: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcu

# Carregando Base de Dados

In [5]:
train_dataset = image_dataset_from_directory(
    path_train,
    image_size=(img_rows, img_cols),
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=True)
train_dev_dataset = image_dataset_from_directory(
    path_train_dev,
    image_size=(img_rows, img_cols),
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=False)

Found 788 files belonging to 7 classes.
Found 106 files belonging to 7 classes.


In [6]:
class_names =train_dev_dataset.class_names
for i in range(len(class_names)):
    class_names[i] = class_names[i].replace('_', ' ')

In [7]:
y_test = np.concatenate([y for x, y in train_dev_dataset], axis=0)

2022-05-10 21:21:53.570556: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-10 21:21:53.591425: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2200000000 Hz


In [8]:
train_dataset = train_dataset.with_options(options)
train_dev_dataset = train_dev_dataset.with_options(options)

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(
    buffer_size=AUTOTUNE)  # Otimização
train_dev_dataset = train_dev_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
      cm (array, shape = [n, n]): a confusion matrix of integer classes
      class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=90)
    plt.yticks(tick_marks, class_names)

    # Compute the labels from the normalized confusion matrix.
    labels = np.around(cm, decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure


def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


In [11]:
run_dir = logs_dir

file_writer_cm = tf.summary.create_file_writer(run_dir + '/cm')


def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(train_dev_dataset)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(
        y_test, test_pred, normalize='true')
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)


# Define the per-epoch callback.
cm_callback = keras.callbacks.LambdaCallback(
    on_epoch_end=log_confusion_matrix)
with strategy.scope():
    # loss
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # métricas
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')]
    data_augmentation = tf.keras.Sequential(
        [tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical')])
    # Keras sequential model with Hyperparameters passed from the argument
    model_pre = tf.keras.models.load_model('model_base/resnet101_plc_full/')
    cnn = Model(model_pre.input, model_pre.layers[-3].output)
    # Para o primeiro treinamento as camadas convolucionais não serão treinadas
    cnn.trainable = False
    # Camadas de Batch Normalization em inference mode
    inputs = keras.Input(shape=input_shape)
    x = cnn(inputs, training=False)
    x = keras.layers.Dropout(0.2)(x)
    predictions = Dense(num_classes)(x)
    model = Model(inputs, predictions)
    lr_denso = 5e-5
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=lr_denso, momentum=0.9), loss=loss, metrics=metrics)


def scheduler_denso(epoch, lr):
    lr_0 = lr_denso
    epoch_ini_lr = 20
    if epoch < epoch_ini_lr:
        learning_rate = lr
        with tf.summary.create_file_writer(run_dir + '/metric').as_default():
            tf.summary.scalar(
                'learning rate', data=learning_rate, step=epoch)
        return learning_rate
    else:
        learning_rate = lr_0 * \
            tf.math.exp(-0.005*(epoch-epoch_ini_lr)).numpy()
        with tf.summary.create_file_writer(run_dir + '/metric').as_default():
            tf.summary.scalar(
                'learning rate', data=learning_rate, step=epoch)
        return learning_rate


callbacks_dense = [tf.keras.callbacks.TensorBoard(run_dir, profile_batch=0, write_graph=False),
                   tf.keras.callbacks.LearningRateScheduler(
    scheduler_denso, verbose=0),
    cm_callback,
    EarlyStopping(patience=15)]
# Training the network
history_dense = model.fit(train_dataset, epochs=n_epochs_ini, verbose=0,
                          validation_data=train_dev_dataset, callbacks=callbacks_dense)

treinamento_denso = len(history_dense.history['loss'])
with strategy.scope():
    cnn.trainable = True
    # loss
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # métricas
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')]
    lr_full = 5e-6
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=lr_full, momentum=0.9),
        loss=loss,
        metrics=metrics
    )


def scheduler_cnn(epoch, lr):
    lr_0 = lr_full
    learning_rate = lr_0 * \
        tf.math.exp(-0.02*(epoch-treinamento_denso)).numpy()
    with tf.summary.create_file_writer(run_dir + '/metric').as_default():
        tf.summary.scalar(
            'learning rate', data=learning_rate, step=epoch)
    return learning_rate

nome_save_model = nome_modelo

model_dir = os.path.join('./model/', nome_save_model)

callbacks_cnn = [tf.keras.callbacks.TensorBoard(run_dir, profile_batch=0, write_graph=False),
                 tf.keras.callbacks.LearningRateScheduler(
                 scheduler_cnn, verbose=0),
                 cm_callback,
                 EarlyStopping(patience=30),
                ModelCheckpoint(filepath=model_dir,monitor="val_loss",save_best_only=True)]

history_cnn = model.fit(train_dataset, epochs=n_epochs, initial_epoch=treinamento_denso, verbose=0,
                        validation_data=train_dev_dataset, callbacks=callbacks_cnn)

treinamento_cnn = len(history_cnn.history['loss'])


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

2022-05-10 21:22:43.556717: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-05-10 21:22:53.821233: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 113 of 256
2022-05-10 21:23:03.700685: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:230] Shuffle buffer filled.
2022-05-10 21:23:05.850602: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


INFO:tensorflow:batch_all_reduce: 418 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 418 all-reduces with algorithm = nccl, num_packs = 1


2022-05-10 21:36:12.169462: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets written to: ./model/resnet_plc_full_transfer/assets
INFO:tensorflow:Assets wr

In [12]:
print(treinamento_denso)
print(treinamento_cnn)

100
78
